In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
!pip install hazm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Embedding, Dropout, GlobalMaxPool1D,MaxPool1D 
from tensorflow.compat.v1.keras.layers import CuDNNGRU

from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam,RMSprop
# Preprocessing
from hazm import *

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Deep/NLP/digimag/fixed_train.csv", sep='	')
test = pd.read_csv("/content/drive/MyDrive/Deep/NLP/digimag/fixed_test.csv", sep='	')
dev = pd.read_csv("/content/drive/MyDrive/Deep/NLP/digimag/fixed_dev.csv", sep='	')

In [ ]:
train=train.append(dev)
train

,Unnamed: 0,content,label,label_id
0,0,نمایش تبلیغ لاک اسکرین گوشی های هواوی تعداد زی...,علم و تکنولوژی,3
1,1,شکست Justice League باکس آفیس بازخوردهای طرفدا...,هنر و سینما,5
2,2,کلاسیک بینی؛ شب اتفاق افتاد فیلم شب اتفاق افتا...,هنر و سینما,5
3,3,اپل سراغ رنده چراکه آپگرید سطل آشغال! نسل کامپ...,علم و تکنولوژی,3
4,4,بررسی جزء جزء Ori and the Blind Forest جعبه اب...,بازی ویدیویی,0
...,...,...,...,...
762,762,۱۰ جزیره زیبای دنیا سفر تماشای ویدیوی دقیقه ای...,سلامت و زیبایی,2
763,763,تریلر بسته الحاقی Iceborne Monster Hunter Worl...,بازی ویدیویی,0
764,764,دیزنی روابطش الکترونیک آرتز می گوید باب ایگر م...,بازی ویدیویی,0
765,765,افتضاح لوت باکس های Battlefront سیاه ترین دوره...,بازی ویدیویی,0


In [ ]:
sentence_train = train['content']
label_train = train['label_id']

print('Number of training sentence: ', sentence_train.shape)
print('Number of training label: ', label_train.shape)

Number of training sentence:  (7663,)
Number of training label:  (7663,)


In [ ]:
from collections import Counter
cnt = Counter(label_train)
cnt = dict(cnt)
print(cnt)

{3: 2495, 5: 1500, 0: 1770, 6: 229, 2: 1449, 4: 108, 1: 112}


In [ ]:
# Convert dataframes to numpy arrays
sentence_train = np.asarray(sentence_train)
label_train = np.asarray(label_train)

In [ ]:
# Train Prepare labels for categorical prediction
categorical_label_train = to_categorical(label_train, 7)
categorical_label_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
import re
normalizer = Normalizer()
lemmatizer = Lemmatizer()

# turn a doc into clean tokens
def clean_data(doc):
    doc = normalizer.normalize(doc) # Normalize document using Hazm Normalizer
    tokenized = word_tokenize(doc)  # Tokenize text
    
    tokens = []
    for token in tokenized:
      token = re.sub("[،:.,;()/+]", " ", token) 
      token = re.sub(r"\!+","!", token)
      token = re.sub(r"\؟+","؟", token)
      token = re.sub(r"\u200c", " ", token)
      tokens.append(token)

    tokens = [w for w in tokens if not len(w) <= 1] # single character removal 
    tokens = [w for w in tokens if not w.isdigit()] # digit remove
    tokens = [lemmatizer.lemmatize(w) for w in tokens] # Lemmatize sentence words using Hazm Lemmatizer

    tokens = ' '.join(tokens)
    return tokens

In [ ]:
# Apply preprocessing step to training data
train_docs = np.empty_like(sentence_train)
for index, document in enumerate(sentence_train):
  train_docs[index] = clean_data(document)

In [ ]:
num_words = 2000

# Create the tokenizer
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_docs)
encoded_docs = tokenizer.texts_to_sequences(train_docs)

In [ ]:
# Find maximum length of training sentences
max_length = max([len(s.split()) for s in train_docs])
print(max_length)

# Pad embeded training sequences
x_train_padded = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

6276


In [ ]:
#vocabulary size
vocab_size = len(tokenizer.word_index)
vocab_size

73976

In [ ]:
model = Sequential()

model.add(Embedding(vocab_size, 300, input_length=max_length))
model.add(CuDNNGRU(300))

model.add(GlobalMaxPool1D())
model.add(Dropout(0.25))
model.add(Dense(300, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

model.summary()
batch_size = 64
epochs = 10

hist = model.fit(x_train_padded, categorical_label_train, validation_split=0.2, batch_size=batch_size, epochs=epochs)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 6276, 300)         22192800  
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (None, 6276, 300)         541800    
_________________________________________________________________
global_max_pooling1d (Global (None, 300)               0         
_________________________________________________________________
dropout (Dropout)            (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               90300     
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 2

In [ ]:
x_test = test['content']
y_test = test['label_id']

print('Number of testing sentence: ', x_test.shape)
print('Number of testing label: ', y_test.shape)

Number of testing sentence:  (852,)
Number of testing label:  (852,)


In [ ]:
# Convert dataframes to numpy arrays
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

In [ ]:
# Applying preprocessing step to test data
test_docs = np.empty_like(x_test)
for index, document in enumerate(x_test):
  test_docs[index] = clean_data(document)

In [ ]:
# Embed testing sequences
encoded_docs = tokenizer.texts_to_sequences(test_docs)
x_test_padded = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [ ]:
categorical_y_test = to_categorical(y_test, 7)

In [ ]:
# Evaluate model
loss, acc = model.evaluate(x_test_padded, categorical_y_test, verbose=1)
print('Test Accuracy: %f' % (acc*100))

27/27 [==============================] - 10s 282ms/step - loss: 0.3421 - accuracy: 0.9354
Test Accuracy: 93.544602


In [ ]:
y_pred= model.predict(x_test_padded)

In [ ]:
text = []
true_label = []
pred_label = []
angry = 0
happy = 0
neutral = 0
for i in range(len(y_pred)):
  text.append(x_test[i])
  true_label.append(y_test[i])
  print(x_test[i])
  print(y_test[i])
  print(np.argmax(y_pred[i]))

صبحانه رژیمی لاغر صبحانه مهم ترین وعده غذایی طول شناخته انرژی نیاز بدن بیدار خواب تامین می کند می دانستید می توانید مصرف وعده سمتی ببرید مصرف صبحانه رژیمی وزن لاغر شوید؟ واقع توجه مواد موجود وعده حیاتی، صبحانه عامل مهم لاغری تبدیل رژیم غذایی متعادلی مصرف می کنید چربی سوزی دارید، وعده ای طولانی غذا نخوردن می خورید، اهمیت زیادی می کند می توان مواد غذایی مناسبی توجه نکته انتخاب وعده صبحانه ۲۵-۳۵ درصد کالری طول سوزانده مصرف صبحانه متعادل حاوی ۴۰۰-۴۵۰ کالری ۲۰۰۰ کالری طول مصرف بخوانید غذاهایی می توان صبحانه خورد کدامند؟ تحقیقی دانشگاه میسوری دانشمندان شد، می دهد مصرف صبحانه هورمون گرلین مواد شیمیایی مغز اشتها ارتباط افزایش می دهد هورمون باعث احساس سیری فرد کاهش گرسنگی، تمایل خوردن خوراکی ساعات آینده دلیل مصرف صبحانه رژیمی متعادل لاغر رسیدن وزن نظرتان منجر صبحانه رژیمی مقوی سرشار مواد مغذی اندازه کافی مصرف بخوانید رژیم غذایی لاغری کاری نیستصبحانه رژیمی شامل ترکیباتی است؟ صبحانه رژیمی مناسب لاغری شامل ترکیبات است:۱۰ ۱۵ گرم چربی۲۵ گرم فیبر۱۳ ۲۰ گرم پروتئینمتخصصان توصیه ویتامین، مواد معدنی کربو

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




5
5
بازی های ۲۰۱۸ دیدگاه منتقدان صنعت بازی سال ۲۰۱۸ قدم پیشرفت لطف شرکت های گرفت، بازی سازان مستقل نقشی سال ۲۰۱۸ شاهد عرضه ی بازی های شگفت انگیزی صحبت بازی های برتر سال ۲۰۱۸ ساده نظر برسد لیستی بازی محدود شود، رتبه وجه کار ساده ای تهیه تنظیم لیستی ۱۰ بازی برتر ۲۰۱۸ رده آن ها، موضوعی شک سلیقه دخالت می کند، نتیجه تصمیم سراغ میانگینی انتخاب منتقدان برویم لیستی تهیه کرده ایم، اساس نمره ی متاکریتیک رتبه است؛ واقع لیست بازی های ۲۰۱۸ توجه نمره ی متاکریتیک قرار گرفته اند مطمئنا می دانید وب سایت متاکریتیک نمرات منتقدان حوزه های سرگرمی، سینما تلویزیون موسیقی بازی جمع آوری میانگینی تهیه می کند نکته ی مهم درباره ی متاکریتیک نشریات منتقدان ضریب متفاوتی میانگین طبق گفته ی متاکریتیک ضریب بررسی تغییر بازی زیر، بازی هایی توانسته اند میانگین نمره ی ۹۰ بالاتر منتقدان دریافت برخورد نمره ای یکسان، اولویت الفبا لحاظ ادامه می توانید لیست بازی های سال ۲۰۱۸ دیدگاه منتقدان صنعت بازی ببینید.۱- Red Dead Redemption ۲میانگین نمرات ۹۷سازنده Rockstar Gamesپلتفرم پلی استیشن ۴، ایکس باکس واندنیایی سازندگان بازی Red De